In [11]:
import numpy as np 
import pandas as pd
import pickle as pkl
import nltk
from rouge import rouge_n_sentence_level

In [12]:
### TEST CASE

summary_sentence = 'the capital of China is Beijing'.split()
reference_sentence = 'Beijing  the capital of China'.split()

# Calculate ROUGE-2.
recall, precision, rouge = rouge_n_sentence_level(summary_sentence, reference_sentence, 2)
print('ROUGE-2-R', recall)
print('ROUGE-2-P', precision)
print('ROUGE-2-F', rouge)

ROUGE-2-R 0.75
ROUGE-2-P 0.6
ROUGE-2-F 0.6666666666666665


In [13]:
import json

with open('half_data_generation.json', 'r') as f:
    x1 = json.load(f)

with open('80_percent_masked.json', 'r') as f:    
    x2 = json.load(f)

In [27]:
def avg_rouge(list1,list2, checklist=False):
    rouge_sum = 0
    for i in range(len(list1)):
        if checklist:
            a = ' '.join(list1[i])
            b = ' '.join(list2[i])
            _ , _, rouge = rouge_n_sentence_level(a, b, 2)
        else:
            _,_,rouge = rouge_n_sentence_level(list1[i],list2[i], 2)
        rouge_sum +=rouge
    rouge_avg = rouge_sum/len(list1)
    return rouge_avg

def get_avg_bleu(list1,list2, checklist=False):
    sumbleu = 0
    for i in range(len(list1)):
        hypothesis = list1[i].split(' ') if not checklist else list1[i]
        reference = list2[i].split(' ') if not checklist else list2[i]
        #the maximum is bigram, so assign the weight into 2 half.
        BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis, weights = (0.5, 0.5))
        sumbleu+=BLEUscore
    avgbleu = sumbleu/len(list1)
    return avgbleu

def get_number_of_ingredient_used(ingredient_list,text, checklist=False):
    ing_count = 0
    for ing in ingredient_list :
        if ing in text :
            ing_count+=1
    ing_frac = ing_count/len(ingredient_list)
    return ing_frac

def get_number_of_ingredient_used2(ingredient_list,text, checklist=False):
    ing_count = 0
    total_words = 0
    for ing in ingredient_list :
        ings = ing.split(' ')
        total_words +=len(ings) 
        for i in ings :
            if i in text :
                ing_count+=1
                
    ing_frac = ing_count/total_words
    return ing_frac

In [30]:
x = []
x.append(x1['goal'])
x.append(x1['ingredients'])
x.append(x1['generated_text'])
x.append(x1['label'])

rouge_avg = avg_rouge(x[2],x[3], checklist=True)
print(np.round(rouge_avg, 4))
bleu_avg = get_avg_bleu(x[3],x[2], checklist=True)
print(np.round(bleu_avg, 4))

ing_frac = [get_number_of_ingredient_used(x[1][i],x[2][i]) for i in range(len(x[1]))]
print(np.round(sum(ing_frac)/len(ing_frac), 4))  

ing_frac = [get_number_of_ingredient_used2(x[1][i],x[3][i]) for i in range(len(x[1]))]
print(np.round(sum(ing_frac)/len(ing_frac), 4))  

0.4222
0.0808
0.1651
0.2376


In [31]:
x = []
x.append(x2['goal'])
x.append(x2['ingredients'])
x.append(x2['generated_text'])
x.append(x2['label'])


rouge_avg = avg_rouge(x[2],x[3], checklist=True)
print(np.round(rouge_avg, 4))
bleu_avg = get_avg_bleu(x[3],x[2], checklist=True)
print(np.round(bleu_avg, 4))

ing_frac = [get_number_of_ingredient_used(x[1][i],x[2][i]) for i in range(len(x[1]))]
print(np.round(sum(ing_frac)/len(ing_frac), 4))  

ing_frac = [get_number_of_ingredient_used2(x[1][i],x[3][i]) for i in range(len(x[1]))]
print(np.round(sum(ing_frac)/len(ing_frac), 4))  

0.4997
0.1186
0.2348
0.4201
